# CS 396 - Differential Privacy Final Project - Kevin Hayes

For this project, my original intent was to try and modify the ResidualPlanner (https://github.com/dkifer/ResidualPlanner) codebase so that it would support using the infinity-norm instead of the l2-norm. Sadly, I have failed in that regard, it's been a few weeks of first understanding the code and associated paper well enough to start modifying it, and I think I've come to understand the paper itself pretty well, but actually changing the underlying mechanism is decidedly beyond my abilities.

So instead of making the modification myself, I've decided to shift my focus to helping summarize the paper and code-base (1) because I think both are very interesting now that I'm understanding them more, and would like to share that with you and (2) so that if someone better equipped, Professor Dong, a graduate student, or a future undergraduate in CS 396 wants to try tackling the problem of getting ResidualPlanner to use the infinity-norm, I hope this document will give them a jumping off point, so that they don't need to start from scratch. With that goal in mind, I'll try to explain some topics/background that I'm sure Professor Dong already knows, but I want this to be a more generally approachable document.

I'm also formatting this as a jupyter notebook so that I can provide interactive examples of residual planner itself to help demonstrate the different parts of the code base. I'm also aiming for a more casual/intuition focused approach to the summary (if you want rigor, you should probably just read the paper itself).

(This document is meant to be run within the residual_planner repository itself, so that the code blocks will actually work correctly).

### Kronecker Product
First, this paper depends heavily on the "Kronecker Product" $\otimes$ which is defined on two matrices, and for every element of the left matrix, it does a scalar/matrix multiplication with the right matrix, and expands the block matrix in place. For example:
$$
\begin{bmatrix}a & b \\ c & d\end{bmatrix} \otimes \begin{bmatrix}e & f \\ g & h\end{bmatrix}
=\begin{bmatrix}
  a\begin{bmatrix}e & f \\ g & h\end{bmatrix} & 
  b\begin{bmatrix}e & f \\ g & h\end{bmatrix} \\ 
  c\begin{bmatrix}e & f \\ g & h\end{bmatrix} & 
  d\begin{bmatrix}e & f \\ g & h\end{bmatrix}
\end{bmatrix}
=\begin{bmatrix}
 a \cdot e & a \cdot f & b \cdot e & b \cdot f\\
 a \cdot g & a \cdot h & b \cdot g & b \cdot h\\
 c \cdot e & c \cdot f & d \cdot e & d \cdot f\\
 c \cdot g & c \cdot h & d \cdot g & d \cdot h
\end{bmatrix}
$$

When thinking about the Kronecker product in this paper, I think it inuitively makes the most sense to think of it as a form of cartesian product, taking every possible pairing of two elements taken from each matrix.


Two important properties of kronecker products which we will use are the "mixed-product" property, which tells us for matrices $A$, $B$, $C$, and $D$:
$$
(A \otimes B)(C \otimes D) = AC \otimes BD
$$

And that the kronecker product is bilinear, particularly the fact that
$$
A \otimes (B + C) = (A \otimes B) + (A \otimes C)
$$

### Attributes
ResidualPlanner often refers to "attributes" as the abstract data which is being stored within the database. Put simply these are best thought of as "enum" from programming languages like C, C++, Java, etc. It is a range of N values which represent something, but ultimately can be represented as a single number.
This is important because it means that the domain for every one of our attributes is precisely defined, and we don't need to worry about the actual meanings behind any attribute.

Something important to note though is that encoding attributes this way does pretty severely limit the types of data which we can store within a data base and still reasonably apply ResidualPlanner to. If we want to allow a "string" field that can be up to 32 ascii characters, then the "attribute" that represents this field would have a domain of size $256^{32} = 2^{256}$ which is too large to ever be reasonably computed with the current system.

Representing all attributes this way is important because if we know the domain of every attribute, then we can represent our database with a "one-hot" encoding. Conceptually, a one-hot encoding is a row vector of length N, where N is the number of possible unique tuples in our database, and we assign each index to a specific tuple. In some ways this is a "brute-force" approach to representing our data, and it seems like this shouldn't ever work, combinatorics are working against us (even if our only attribute is a single 32-character string as mentioned before, then $N=2^{256}$). It's actually useful for two reasons though, first, our vectors here are almost always extremely sparse, so we can encode long strings of zeros much more efficiently, and second, because when concerning privacy, fields like arbitrary strings are rare. You want the data to be anonymous in the first place, but if we actually have a large number of unique strings in our database, then they can be used as a UUID, and we can't exactly add noise to a string.

Then we can represent a single tuple in our database as an N length vector with a single 1, in the index corresponding to the tuples specific value. And then then the entire database can be represented as the sum of all of these vectors, meaning that for duplicates in our database we just keep the number of duplicates.

One important thing to note, is that we can use kronecker products to generate these "one-hot"encodings in the first place. If we take each attribute, and encode it's values as the unit vectors with a single 1 and all other elements 0, of some $|Attr|$ sized vector space, then a specific row can be encoded as the kronecker product
$$
\text{one-hot-encode-row}(row) = \bigotimes_{attr \in D} \text{one-hot-encode-attribute}(row_{attr})
$$
which will result in a one-hot encoded version of our row.

We will need to impose some order on the attributes as well, because the kronecker product is not associative, and for all kronecker products of the form $\bigotimes_{attr \in D}$, we will use the same order.

### Workloads

The paper represents all queries on our database as "counts" of some sort. As we saw in class, this isn't too much of a problem, and many more complicated types of queries can be represented as counts, because we could issue the "count" of a specific row(s) in the database to make arbitrary queries.

Specifically a query is a set of attribute(s) or the null-set
$$
\emptyset,\\
\{Attr_1\},\\
\{Attr_1, Attr_2\}
$$
Where the null-set indicates the count of the entire database, and a query with X attributes would compute the cartesian product of every attributes domain in the set, and then for every element in the cartesian set, return the count of rows in the database which have those values. In this way, then a query containing every attribute in our database, should return the "sum of one-hot encodings" database itself.

### Questions Answered by Each Query

So far all we have seen is terminology, but we have enough now to look at one of the first really interesting thoughts which the paper has, that we can extract out the minimal amount of extra information each query has from it's subqueries.

To help explain, consider a database with contains the attribute "IsAlive" whose domain is $\{True,False\}$, then consider the query
$$
\{IsAlive\}
$$

This should give us two numbers in response, the number of rows in our database with "True" (how many people are alive?) and the number of rows with "False" (how many people are dead?). In addition to those questions, we can notice that we can also derive the query to the question: "How many people are there total?" but adding up both numbers. This query is represented by the null-set $\emptyset$.

Importantly from this, the paper notes that from the answer to any query $Q$, we can derive the answer to the query represented by any subset of $Q$.
So if we answer the query
$$
\{Attr_1,Attr_2\}
$$

We've also implicitly answered the queries:
$$
\{Attr_1\}, \{Attr_2\}, \emptyset
$$

This is extremely important for keeping consistent answers, because if both $\{IsAlive\}$ and $\emptyset$ are submitted as queries, then we need to make sure that after adding noise that they both agree on the total number of rows in the dataset.

Really, the only additional information provided by $\{IsAlive\}$ if we already know $\emptyset$ is the relative values of each value of the attribute. Trying to model that additional information, is what motivates creating the idea of "subtraction" matrices.

### Subtraction Matrices

ResidualPlanner defines the new term "subtraction" matrices, of the form $S_i$ where the $i$-th has $i-1$ rows and $i$ columns, where the leftmost column is all ones, and the rest of the rows form $-I_{i \times i}$ so for example:

$$
S_2 = \begin{bmatrix} 1 & -1 \end{bmatrix}\\
$$
$$
S_3 = \begin{bmatrix} 1 & -1 & 0 \\ 1 & 0 & -1 \end{bmatrix}
$$
$$
S_4 =
\begin{bmatrix}
1 & -1 & 0 & 0\\
1 & 0 & -1 & 0\\
1 & 0 & 0 & -1\\
\end{bmatrix}
$$

And we define $S_0$ to be $\begin{bmatrix}1\end{bmatrix}$.

The paper introduces and uses subtraction matrices, before they've actually given any form of motivation behind why they take the form they do, and what they are actually useful for. Which meant it took me far too long to wrap my head around them.

If we multiply the $i$ long vector of answers to a query by the corresponding $S_i$ subtraction matrix, then we only end up with the differences between the first entry and all the rest. Effectively "losing" the information about the absolute count, and only leaving behind the relative values.

### Residual Matrices

It would be really nice if we could just stop here, and use subtraction matrices on their own, in order to extract the additional information we get as more attributes are added to a query, but it's not quite so simple. We actually need another concept to make it all work: Residual Matrices.

For every $Q$ query we have, we will end up multiplying our dataset $x$ by a corresponding residual matrix $R_Q$, adding noise, giving us $R_Qx + N$ where $N$ is our noise.
Then once we have all of these noisy answers, we can recombine them, expecting that each noisy answer is independant of one another, ($R_\emptyset$ encodes the noisy total count, and $R_{\{Attr\}}$ encodes the noisy relative values between all the possible values of $Attr$ but not the total count).
Thus if all of them are independant, we can recontruct answers to our original queries, without any contradictions.

(A small nuance here is that we actually need to compute $R_Qx + N$ for the closure of all subqueries, e.g. if we have queries $\emptyset$ and $\{Attr_1,Attr_2\}$ then we still need to compute residuals for $\{Attr_1\}$ and $\{Attr_2\}$ to allow us to reconstruct the answer to $\{Attr_1,Attr_2\}$).

They're defined for some query $Q$ on an attribute domain $D$:
$$
R_{Q} = \bigotimes_{attr \in D} \begin{cases}S_{|attr|} & attr \in Q\\\textbf{1}^{|attr|} & attr \notin Q\end{cases}
$$

I think this formulation is pretty hard to wrap your head around, so I want to spend a bit of time going through some examples to help give an intuition for how they work.

Consider the case of the empty "count" query:
$$
R_{\emptyset} = \bigotimes_{attr \in D} 1^{|attr|} = 1^{\prod_{attr \in D} |attr|} = 1^{|x|}
$$

Because $attr \notin \emptyset$ will always be true. In this case, our residual matrix will always return a single number count of rows in $R_\emptyset x = 1^{|x|}x = \sum_{x_i \in x} x_i$.
This is basically what we expect, because there are no proper subsets of $\emptyset$.

Trying a more complicated set, we will actually use the code-base provided for us

In [16]:
from class_resplan import *

# Interestingly their codebase actually generates subtraction matrices of a different form,
# with a diagonal of all 1's and the next diagonal up all -1's.
#
# From what we talk about above, this should still work, it removes all absolute information only leaving relative parts.
# But it is notably different from the layout described in the actual paper, which focuses on the first column.
# (I can't find any changes to the effect of this though)

# Feel free to change this and see how it generates larger matrices
i = 5
subtract_matrix(i, False) # The "False" is to avoid generating a sparse matrix

array([[ 1., -1.,  0.,  0.,  0.],
       [ 0.,  1., -1.,  0.,  0.],
       [ 0.,  0.,  1., -1.,  0.],
       [ 0.,  0.,  0.,  1., -1.]])

In [17]:
# let's pregenerate some subtraction matrices
S = [subtract_matrix(i,False) for i in range(1,6)]

# Let's play around with a simple domain
# We have 3 attributes, ("isAlive" can either be True or False, "hasDisease" can be Yes or No, and "numberOfEars" can be 0, 1, or 2.
attributes = {"isAlive" : [True,False], "hasDisease" : ["Yes","No"], "numberOfEars" : [0,1,2]}

# The code-base never uses an actual kronecker product because if you know you be doing something like:
# kron(A,B) * vector, theres a faster way to do it using properties of the kronecker product.
# So we'll need to use np.kron if we want to visualize stuff

def residual_matrix(query, attrs):
    res_mat = None
    for name,domain in attrs.items():
        cur_mat = None
        if name in query:
            cur_mat = subtract_matrix(len(domain),False)
        else:
            cur_mat = [1] * len(domain)
        if res_mat is None:
            res_mat = cur_mat
        else:
            res_mat = np.kron(res_mat,cur_mat)
    return res_mat
            
query = set(["isAlive","hasDisease"])
residual_matrix(query, attributes)

array([[ 1.,  1.,  1., -1., -1., -1., -1., -1., -1.,  1.,  1.,  1.]])

In [18]:
# This allows us to name numbers so we can see what the results of multiplying by the matrix is
# It looks really complicated because there's some additional rules to help with the printing (avoiding double negatives, stuff like that)
# But really it just helps track what happens to a value over time, so we can see what field is semantically after mutliplying matrices
class TracedNumber:
    def __init__(self, n, name):
        self.n = n
        self.name = name
    def __add__(self, other):
        if isinstance(other, TracedNumber):
            if other.name[0] == '-':
                return TracedNumber(self.n + other.n, "(" + self.name + ")-(" + other.name[1:] + ")")
            else:
                return TracedNumber(self.n + other.n, "(" + self.name + ")+(" + other.name + ")")
        else:
            if other == 0:
                return TracedNumer(self.n, self.name)
            else:
                if other > 0:
                    return TracedNumber(self.n + other, "(" + self.name + ")+(" + str(other) + ")")
                else:
                    return TracedNumber(self.n + other, "(" + self.name + ")-(" + str(-other) + ")")
    __radd__ = __add__
    def __sub__(self, other):
        return self + (other * (-1.0))
    __rsub__ = __sub__
    def __mul__(self,other):
        if isinstance(other, TracedNumber):
            return TracedNumer(self.n * other.n, "(" + self.name + ")*(" + other.name + ")")
        else:
            if other == 1:
                return TracedNumber(self.n, self.name)
            elif other == -1:
                if self.name[0] == '-':
                    return TracedNumber(-self.n, self.name[1:])
                else:
                    return TracedNumber(-self.n, "-"+self.name)
            else:
                return TracedNumber(self.n * other, "(" + self.name + ")*(" + str(other) + ")")
    __rmul__ = __mul__
    def __str__(self):
        return "[" + str(self.n) + "," + self.name + "]"

In [19]:
# Here's a data set to play around with
rows = [
    {"isAlive":False,"hasDisease":"No","numberOfEars":2},
    {"isAlive":False,"hasDisease":"Yes","numberOfEars":2},
    {"isAlive":False,"hasDisease":"Yes","numberOfEars":2},
    {"isAlive":False,"hasDisease":"No","numberOfEars":1},
    {"isAlive":True,"hasDisease":"No","numberOfEars":2},
    {"isAlive":True,"hasDisease":"Yes","numberOfEars":2},
    {"isAlive":True,"hasDisease":"No","numberOfEars":0},
    {"isAlive":False,"hasDisease":"Yes","numberOfEars":1},
    {"isAlive":True,"hasDisease":"No","numberOfEars":2},
]

# Encode it as a sum of one-hot vectors
def encode_dataset(rows, attributes):
    import functools
    encoded_dataset = np.array([0] * functools.reduce(lambda x,y: x*y, [len(domain) for domain in attributes.values()]))
    for row in rows:
        encoded_row = None
        for name,value in row.items():
            domain = attributes[name]
            index = domain.index(value)
            encoded_attr = [0] * len(domain)
            encoded_attr[index] = 1
            if encoded_row is None:
                encoded_row = encoded_attr
            else:
                encoded_row = np.kron(encoded_row,encoded_attr)
        encoded_dataset += encoded_row
    return encoded_dataset
        
encoded_dataset = encode_dataset(rows, attributes)

# Trace what each number in the encoded data base actually represents, so we can see some semantic meaning in the output
def trace_encoded_dataset(encoded_dataset, attributes):
    encoded_dataset=encoded_dataset.tolist()
    domains = [domain for domain in attributes.values()]
    index = 0
    for choice in itertools.product(*domains):
        encoded_dataset[index] = TracedNumber(encoded_dataset[index],str(choice))
        index += 1
    return np.array(encoded_dataset)
encoded_dataset = trace_encoded_dataset(encoded_dataset, attributes)

# Useful for printing np.array's of TraceNumbers
str_array = np.vectorize(str)
str_array(encoded_dataset)

array(["[0,(True, 'Yes', 0)]", "[0,(True, 'Yes', 1)]",
       "[1,(True, 'Yes', 2)]", "[1,(True, 'No', 0)]",
       "[0,(True, 'No', 1)]", "[2,(True, 'No', 2)]",
       "[0,(False, 'Yes', 0)]", "[1,(False, 'Yes', 1)]",
       "[2,(False, 'Yes', 2)]", "[0,(False, 'No', 0)]",
       "[1,(False, 'No', 1)]", "[1,(False, 'No', 2)]"], dtype='<U21')

In [20]:
# Let's try a simpler database
attributes = {"isAlive":[True,False],"hasDisease":[True,False]}
rows = [
    {"isAlive":True,"hasDisease":True},
    {"isAlive":True,"hasDisease":False},
    {"isAlive":False,"hasDisease":True},
]
x = encode_dataset(rows,attributes)

# Add tracing so we can see what happens when we play around with the numbers
x = trace_encoded_dataset(x, attributes)

# The List of Queries
Q = [
    [],
    ["isAlive"],
    ["hasDisease"],
    ["isAlive", "hasDisease"],
]

residual_matrices = {}
residuals = {}
for query in Q:
    residual_matrices[str(query)] = residual_matrix(query, attributes)
    residuals[str(query)] = residual_matrices[str(query)] @ x

print("Residual Matrices:")
for query,matrix in residual_matrices.items():
    print("\t", query, " : ", str_array(matrix))

print("Residuals:")
for query,residual in residuals.items():
    print("\t", query, " : ", str_array(residual))

Residual Matrices:
	 []  :  ['1' '1' '1' '1']
	 ['isAlive']  :  [['1.0' '1.0' '-1.0' '-1.0']]
	 ['hasDisease']  :  [['1.0' '-1.0' '1.0' '-1.0']]
	 ['isAlive', 'hasDisease']  :  [['1.0' '-1.0' '-1.0' '1.0']]
Residuals:
	 []  :  [3,((((True, True))+((True, False)))+((False, True)))+((False, False))]
	 ['isAlive']  :  ['[1,((((True, True))+((True, False)))-((False, True)))-((False, False))]']
	 ['hasDisease']  :  ['[1,((((True, True))-((True, False)))+((False, True)))-((False, False))]']
	 ['isAlive', 'hasDisease']  :  ['[-1,((((True, True))-((True, False)))-((False, True)))+((False, False))]']


## Reconstruction

From our marginals, (noise added or not), we then need to be able to reconstruct answers to the original queries which were submitted. Because of a very useful property of Kronecker products this isn't too bad though, because for both inverses and pseudo inverses,
$$
(A \otimes B)^{-1} = A^{-1} \otimes B^{-1}
$$

For this paper we actually only really care about "pseudo inverses" $A^+$, which have the properties
$$
\begin{align*}
(AA^+)A &= A\\
A^+(AA^+) &= A^+\\
(A^+A)^* &= A^+A\\
(AA^+)^* &= AA^+\\
\end{align*}
$$

We can calculate the pseudo-inverse of a subtraction matrix fairly easily, and so we can define a matrix $U$ such that

$$
U_Q = \bigotimes_{attr \in D} \begin{cases}S_{|attr|}^{-1} & attr \in Q\\ \frac{1}{|attr|}\textbf{1}^{|attr|} & attr \notin Q\\ \end{cases}
$$

And because the kronecker product of inverses is the inverse of the kronecker product, then $U_Q$ can be used as our pseudo-inverse to $R_Q$.
So we can compute the residuals $R_q x$ for all subqueries $q \in Q$, and add noise to them.
Then if we compute $U_q$, we can "invert" by computing $U_q(R_q x + N)$. The sum of all these "inverted" values will then be our final noisy answer to the original query $Q$.

Because if we have multiple queries $Q_1$, $Q_2$, ..., $Q_n$, then we just keep a set of sets for all of them. Then if there is any sub-query $q$ such that $q \in Q_1$ and $q \in Q_2$, we only compute it's residual once, and invert it once, and save the value. Then we can use it in reconstructing $Q_1$ and $Q_2$ (or as many more $Q$ as we have), which saves us work, and means that our answers will be consistent.

## Noise

So far, we've pretty much ignored the problem of adding noise. I've come to the conclusion that it's somewhat orthogonal to the general issue of the matrix mechanism (consistency, reducing redundant computation) but the authors spend a decently large amount of time on it, so I didn't want to completely skip over it. It is also important if someone in the future wants to get ResidualPlanner to use the infinity-norm, because it's during the noise adding process that it comes up.

Adding the noise becomes an optimization problem, with a trade-off between the amount of privacy added, and the amount of accuracy lost from the noise.
We can view the problem from two different perspectives, depending on whether we select an accuracy level and then try optimize for as much privacy as possible, or if we set a minimum privacy level, and then try to reduce the innaccuracy as much as possible.

Generally we will be setting a "privacy budget" and then optimizing to be as accurate as possible while staying within that budget. Accuracy is modelled through the variance $\sigma^2$ that applying some noise to each residual value will have on the answers which we compute to the final answer. Actually optimizing $\sigma^2$ can be done with a linear programming solver, but the exact formulation depends on the cost function we select. Either minimizing the maximum variance, or minimizing the l2-norm of all variances. To be honest, I'm slightly conflicted after understand the paper more about the difference between optimizing for the "maximum-variance" and optimizing the infinity-norm of the variances. I'm guessing there's a subtle difference I'm not picking up on, and I know that the infinity-norm isn't actually just the maximum, but practically speaking it seems like you would expect the same results to me.

# Conclusion

I just wanted to say here at the end, thank you professor. I've had a great time in this class, and I think I've gotten a pretty solid foundations on the math behind differential privacy, and math in general. I hope this report is actually of some help, but I'm not 100% sure that it will be much easier to understand than the paper itself. After trying to explain these concepts myself (even after I got a solid grasp on them) I have a much higher degree of respect for how hard it is to communicate these topics (to any future students, if you want to read the paper itself, read Appendix B before anything else, it will be much much easier).